###These files are provided by the Neo4j AuraDB free instance, under Connect with Python chosen out of the provided languages. My intent of recreating them was to get a feel for all of the requirements of a neo4j program, including uri and driver connection, authentication, incorporating cypher, and more.

##neo4j_practice.py

The **\_\_init__** function defines the driver using the uri and authentication provided as arguments. The uri connection link was provided by the AuraDB free instance, and the authentication was provided when I created my AuraDB account. 
```
self.driver = GraphDatabase.driver(uri, auth=(user, password))
```

You must remember to close the driver after use, so a **close** function was created to make this step simple. 
```
self.driver.close()
```

The **main** function initializes the uri, username, and password, here my password is located in a separate file located on my personal machine. It then calls the App class to connect to the uri and driver. Multiple functions, explained later in the notebook, were also called for practice, and finally the driver was closed. 
```
app = App(uri, user, password)
app.find_person("Katie")
app.close()
```







####The next set of functions are basic examples of those that could be found in an API. They showcase how to incorporate Cypher queries into a python program. 



**create_and_return_friendship** starts a driver session object and writes to the transaction which allows the driver to handle errors and retries. It calls to the helper function **\_create_and_return_friendship** that holds the query, runs it, and returns the result. The original function prints the resulting message telling a relationship between the two provided Persons was created. 



```
    def create_friendship(self, person1_name, person2_name):
        with self.driver.session() as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(
                self._create_and_return_friendship, person1_name, person2_name)
            for row in result:
                print("Created friendship between: {p1}, {p2}".format(p1=row['p1'], p2=row['p2']))
```



```
    @staticmethod
    def _create_and_return_friendship(tx, person1_name, person2_name):
        query = (
            "CREATE (p1:Person { name: $person1_name }) "
            "CREATE (p2:Person { name: $person2_name }) "
            "CREATE (p1)-[:KNOWS]->(p2) "
            "RETURN p1, p2"
        )
        result = tx.run(query, person1_name=person1_name, person2_name=person2_name)
        try:
            return [{"p1": row["p1"]["name"], "p2": row["p2"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise
```





**find_person** holds the same structure as the function described above, it creates a driver session, calls its helper function **\_find_and_return_person** who holds the query, runs it, and returns the resulting information, and the originial function again prints to respective message if the Person was found. 

**remove_all_persons_and_relationships**, again, holds the same structure by creating driver session, calling helper function **\_find_and_remove_all** which runs the corresponding query and returns the information. The first function then prints the message that the person and its relationships were removed. 

##with_arguments.py

This file holds the same architecture as the previous file. The only difference is that the user can provide arguments when running the program. These arguments are used to decipher between dealing with Movie nodes or Person nodes. Movie and Person are the *label* argument of the node, and the name of the person or the title of the movie is the *name* argument of the node. 

A **parse_all_args function** was created to parse the users arguments, and in **main**, the App class and its functions are all called with the name and label arguments passed to them. 



```
    name = args.name
    label = args.label
    app = App(uri, user, password, name, label)
    app.find_node(label, name)
```



```
    #example run: ./with_arguments.py -label Person -name Katie
```



Here is an example of the **create_node** function now that it uses arguments.



```
    def create_node(self, label, name):
        with self.driver.session() as session:
            result = session.read_transaction(self._create_and_return_node, label, name)
            for row in result:
                print("Created ", label, ": {row}".format(row=row))
```




```
    @staticmethod
    def _create_and_return_node(tx, label, name):
        if label == 'Person':
            query = (
                "CREATE (p:Person {name: $name})"
                "RETURN p.name AS name"
            )
        if label == 'Movie':
            query = (
                "CREATE (m:Movie {title: $name})"
                "RETURN m.title AS name"
            )
        result = tx.run(query, name=name)
        return [row["name"] for row in result]

    def find_node(self, label, name):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_and_return_node, label, name)
            if len(result) == 0:
                print("Could not find ", label,": ", name)
            else:
                for row in result:
                   print("Found ", label,": {row}".format(row=row))
```

